<a href="https://colab.research.google.com/github/Nebiyu01/NuclearSpecklesQuantification/blob/main/MLProject_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
!pip install vaderSentiment

In [41]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [42]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
data = pd.read_csv('Tesla.csv')

coulmns_to_keep = ['tweet', 'language', 'date', 'nlikes', 'nreplies', 'nretweets']

cleaned_data = data[coulmns_to_keep]

print(cleaned_data.head())

cleaned_data.to_csv('cleaned_Tesla.csv', index=False)

                                               tweet language  \
0  @GailAlfarATX @elonmusk @Tesla @teslacn @Tesla...       en   
1  @elonmusk @GailAlfarATX @Tesla @teslacn @Tesla...       en   
2  @elonmusk #Think about buying a country , #Mex...       en   
3  @get_innocuous Actual receipts, and yet you ha...       en   
4  Tesla wall battery for the save! Power went ou...       en   

                  date  nlikes  nreplies  nretweets  
0  2022-07-11 17:06:24       0         0          0  
1  2022-07-11 17:06:21       0         0          0  
2  2022-07-11 17:06:20       0         0          0  
3  2022-07-11 17:06:12       0         0          0  
4  2022-07-11 17:06:09       0         0          0  


In [44]:
# Handle missing values
cleaned_data = cleaned_data.dropna(subset=['tweet']).copy()  # Explicitly create a copy to avoid SettingWithCopyWarning

# Tokenization, stopword removal, and text normalization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [45]:
def preprocess_text(text):
    # Remove URLs, mentions, hashtags, and special characters
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r"@\w+", "", text)  # Remove mentions
    text = re.sub(r"#\w+", "", text)  # Remove hashtags
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # Remove special characters and numbers

    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    return " ".join(tokens)

In [47]:
cleaned_data['cleaned_tweet'] = cleaned_data['tweet'].apply(preprocess_text)

In [48]:
# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

In [49]:
def get_sentiment(text):
    scores = sia.polarity_scores(text)
    if scores['compound'] >= 0.05:
        return 'Positive'
    elif scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'


In [50]:
cleaned_data['sentiment'] = cleaned_data['cleaned_tweet'].apply(get_sentiment)
print(cleaned_data[['tweet', 'cleaned_tweet', 'sentiment']].head())

                                               tweet  \
0  @GailAlfarATX @elonmusk @Tesla @teslacn @Tesla...   
1  @elonmusk @GailAlfarATX @Tesla @teslacn @Tesla...   
2  @elonmusk #Think about buying a country , #Mex...   
3  @get_innocuous Actual receipts, and yet you ha...   
4  Tesla wall battery for the save! Power went ou...   

                                       cleaned_tweet sentiment  
0  six still live home homeschooled taught selfed...  Negative  
1                            go dozen kid missing go  Negative  
2  buying country could turn richest country worl...  Positive  
3  actual receipt yet havent asked anyone buy tes...  Positive  
4    tesla wall battery save power went still run ac  Positive  


In [51]:
cleaned_data.to_csv('sentiment_analyzed_Tesla.csv', index=False)
